# Introduction to <font color= #b30047>[Keras](https://keras.io/)</font> using the *Font type Recognition Example*

<img src="https://s3.amazonaws.com/keras.io/img/keras-logo-2018-large-1200.png" alt="Keras logo" height="100" width="250"> 

---

<font size=4 >Summer Seminar:</font> <font size=4 color= orange>Practical Introduction to Deep Learning & Keras</font>

 <img src="https://pbs.twimg.com/profile_images/969243109208018946/w2GzDfiC_400x400.jpg" alt="IPTC" height="50" width="50"> 
 ## * [IPTC](https://iptc.upm.es/) and [MSTC](http://mstc.ssr.upm.es/big-data-track)
 
---
---


## <font color= #00cc00>Next cells load the Font Type dataset:</font>


In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


"""
Load and data
"""

import os
from six.moves import urllib

file_url = 'https://github.com/bloolizard/PlayWithTensorFlow/raw/master/data_with_labels.npz'
file_name = 'data_with_labels.npz'

if not os.path.exists(file_name):
    urllib.request.urlretrieve(file_url, file_name)
    
    
# Load data
data = np.load('data_with_labels.npz')

train = data['arr_0']/255.
labels = data['arr_1']



---

<font size=4 color=red> PLEASE BE AWARE OF HOW CRITICAL IS TO NORMALIZE INPUTS TO Neural Networks !! </font>

---



## One-Hot-Encoding of labels using Keras (see https://keras.io/utils/#to_categorical)

In [0]:
from keras.utils import to_categorical
onehot= to_categorical(labels, num_classes=len(np.unique(labels)))


### Permutation! + split data into training / validation

In [0]:
 
# Split data into training (90%) and validation (10%)
np.random.seed(100)

indices = np.random.permutation(train.shape[0])

valid_cnt = int(train.shape[0] * 0.1)

test_idx, training_idx = indices[:valid_cnt],\
                         indices[valid_cnt:]
  
test, train = train[test_idx,:],\
              train[training_idx,:]
  
onehot_test, onehot_train = onehot[test_idx,:],\
                        onehot[training_idx,:]
  

# Reshape to represent each image as a vector

In [0]:
train=train.reshape([-1,train.shape[1]*train.shape[2]])
test=test.reshape([-1,test.shape[1]*test.shape[2]])

print('Train shape=', train.shape , '\nTest shape=', test.shape)

In [0]:
print('Train shape=', train.shape , '\nTest shape=', test.shape)

# [Keras](https://keras.io/) Model


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD


model = Sequential()

model.add(Dense(128, activation='relu', input_dim=train.shape[1]))
# Dense(128) is a fully-connected layer with 128 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 1296-dimensional vectors.

model.add(Dense(32, activation='sigmoid'))


model.add(Dense(onehot_train.shape[1], activation='softmax'))
## onehot_train.shape[1] is the number of classes


model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),
              metrics=['accuracy'])

model.summary()

### ... adding more layers is easy (try uncommenting options in this cell)

In [0]:
'''

# Use 300 epochs
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD


model = Sequential()

model.add(Dense(1024, activation='relu', input_dim=train.shape[1]))
# Dense(128) is a fully-connected layer with 128 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 1296-dimensional vectors.

model.add(Dense(512, activation='sigmoid'))

model.add(Dense(512, activation='sigmoid'))

model.add(Dense(512, activation='sigmoid'))

model.add(Dense(32, activation='sigmoid'))

model.add(Dense(onehot_train.shape[1], activation='softmax'))
## onehot_train.shape[1] is the number of classes


model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),
              metrics=['accuracy'])

model.summary()
'''

## Train the model: 
- ## we now use history= ....
- ## and train / validation (test?) data

In [0]:

history=model.fit(train, onehot_train,
         epochs=100,
         batch_size=128,
         validation_data=(test, onehot_test),
         verbose=1)

In [0]:
print(history.history.keys())

In [0]:
print('Train Accuracy: ',np.round(history.history['acc'][-1],2))
print('Test_Accuracy: ',np.round(history.history['val_acc'][-1],2))

# Plot the accuracy curves
plt.plot(history.history['acc'],'bo')
plt.plot(history.history['val_acc'],'rX')
plt.grid()

In [0]:
# Plot the accuracy curves
plt.plot(history.history['loss'],'bo')
plt.plot(history.history['val_loss'],'rX')
plt.grid()

## Confusion Matrix

In [0]:
pred_probabilities= model.predict(test)

print('First Five Probs.:\n',pred_probabilities[0:5])

print('\n\nFirst Five Classes:\n',onehot_test[0:5])

In [0]:
!pip install pandas_ml

In [0]:
from pandas_ml import ConfusionMatrix

ConfMatrix=ConfusionMatrix(np.argmax(onehot_test,1), np.argmax(pred_probabilities,1))

ConfMatrix.plot(normalized=True,backend='seaborn')

In [0]:
import seaborn as sns

from sklearn.metrics import confusion_matrix as cm

ConfMatrix=cm(np.argmax(onehot_test,1), np.argmax(pred_probabilities,1))

print('Confusion Matrix:\n',ConfMatrix)

ax= plt.subplot()
sns.heatmap(ConfMatrix, annot=True, ax = ax); #annot=True to annotate cells

# labels, title and ticks
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')

ax.xaxis.set_ticklabels(['FT_1', 'FT_2','FT_3','FT_4','FT_5'])
ax.yaxis.set_ticklabels(['FT_1', 'FT_2','FT_3','FT_4','FT_5']);